In [ ]:
# %matplotlib widget
import csv
import pandas as pd
import numpy as np
import torch
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import os
import matplotlib.image as mpimg
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
#import mpld3
from PIL import Image
import random


#%pip install plotly
#%pip install pandas
#%pip install mpld3
#%pip install -U scikit-learn
#%pip install ipympl

In [ ]:
folder = '../RESULTS'
file_names_path=os.path.join(folder,'checkpoint.txt')
f = open(file_names_path, 'r')
file_names = f.read()
f.close()

In [ ]:
print(len(file_names))

In [ ]:
n_images = 12273

In [ ]:
int_rep_path=os.path.join(folder,'internal_rep.pt')
rgb_int_rep_path=os.path.join(folder,'rgb_internal_rep.pt')
ir_int_rep_path=os.path.join(folder,'ir_internal_rep.pt')

int_rep=torch.load(int_rep_path)[:n_images]
rgb_int_rep=torch.load(rgb_int_rep_path)[:n_images]
ir_int_rep=torch.load(ir_int_rep_path)[:n_images]

In [ ]:
#int_rep=int_rep.cpu().detach().numpy()

In [ ]:
print(int_rep.shape)
print(rgb_int_rep.shape)
print(ir_int_rep.shape)

In [ ]:
int_rep=torch.cat((int_rep,rgb_int_rep,ir_int_rep), 0)

In [ ]:
%%time
# This takes 10 minutes on entire dataset (12273)
embedd_int_rep = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=3).fit_transform(int_rep)

In [ ]:
print(embedd_int_rep.shape)

In [ ]:
plt.scatter(embedd_int_rep[:,0], embedd_int_rep[:,1], alpha=0.5, color='black', s=0.2)
plt.show()

In [ ]:
# dim=12273
dim = n_images
rgb_ir = embedd_int_rep[:dim,:]
rgb = embedd_int_rep[dim:dim+dim,:]
ir = embedd_int_rep[dim+dim:dim+dim+dim,:]
plt.scatter(rgb_ir[:,0], rgb_ir[:,1], alpha=0.5, color='blue', s=5, label=f'RGB + IR')
plt.scatter(rgb[:,0], rgb[:,1], alpha=0.5, color='red', s=5, label=f'RGB')
plt.scatter(ir[:,0], ir[:,1], alpha=0.5, color='green', s=5, label=f'IR')
plt.title('Scatter Plot')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.show()

In [ ]:
output_path=os.path.join(folder,'output.csv')

In [ ]:
classification=[]
image_paths=[]
with open(output_path, newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        # print(row)
        classification.append(row)
        image_paths.append("." + row[0][:-1])

In [ ]:
len(classification)

In [ ]:
print(rgb_ir.shape)
print(rgb.shape)
print(ir.shape)

In [ ]:
# Assuming rgb.shape is (2580, 2) and len(classification) is 2580
#x = rgb_ir[:, 0]  # Get the first column of rgb (x-coordinates)
#y = rgb_ir[:, 1]  # Get the second column of rgb (y-coordinates)
x = rgb[:, 0]  # Get the first column of rgb (x-coordinates)
y = rgb[:, 1]  # Get the second column of rgb (y-coordinates)
#x = ir[:, 0]  # Get the first column of rgb (x-coordinates)
#y = ir[:, 1]  # Get the second column of rgb (y-coordinates)

print("x.shape", x.shape)
print("y.shape", y.shape)

In [ ]:
# Create a scatter plot
plt.figure(figsize=(8, 6))  # Set the figure size
colors = ['black', 'r']  # Define colors for the classes (red for 0, green for 1)


In [ ]:
Classification=[]
for Class in classification:
    #print(Class[1])
    Classification.append(Class[2])

In [ ]:
# Iterate over the unique labels and plot the points with corresponding colors
for label in [0, 1]:
    indices = [i for i, l in enumerate(Classification) if int(l[0]) == label]
    plt.scatter(x[indices], y[indices], c=colors[label], label=f'Class {label}', alpha=0.5, s=5)

plt.title('Scatter Plot')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.show()

In [ ]:
z = np.stack((x, y), axis=1)
z.shape

In [ ]:
v=np.array([0, 0])
# v=np.array([-71.0,88.0])
#v=np.array([-102.9,-46.0])

# Step 1: Broadcast V to match the shape of C
v_broadcasted = np.tile(v, (z.shape[0], 1))

# Step 2: Calculate the Euclidean distances
distances = np.linalg.norm(z - v_broadcasted, axis=1)

# Step 3: Find the indices of the 10 smallest distances
closest_indices = np.argsort(distances)[:10]

print(closest_indices)

In [ ]:
# Create a figure with a grid of subplots
num_rows = 5
num_cols = 2
fig, axes = plt.subplots(num_rows, num_cols)#, figsize=(20, 8))

for i, index in enumerate(closest_indices):
    row = i // num_cols
    col = i % num_cols
    print(classification[index])
    image_path=classification[index][0][:-1]
    image = mpimg.imread("." + image_path)
    axes[row, col].imshow(image)
    axes[row, col].axis('off')

# Adjust spacing between subplots
plt.subplots_adjust(wspace=0.1, hspace=0.1)

# Display the figure
plt.show()


In [ ]:
# Iterate over the unique labels and plot the points with corresponding colors
fig=plt.figure()
for label in [0, 1]:
    indices = [i for i, l in enumerate(Classification) if int(l[0]) == label]
    plt.scatter(x[indices], y[indices], c=colors[label], label=f'Class {label}', alpha=0.5, s=5)


plt.scatter(x[closest_indices], y[closest_indices], c='blue', alpha=0.5, s=15)

plt.title('Scatter Plot')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.show()

In [ ]:
# Generate data x, y for scatter and an array of images.
images=[]
for i, image_path in enumerate(image_paths):
    
    if i < 10:
        print(image_path)
    try:
        image = Image.open(image_path)
        if i < 10:
            print(image.size)
        # images.append(image.resize((int(image.size[0]/10), int(image.size[1]/10))))
    except FileNotFoundError:
        pass


In [ ]:
# # create figure and plot scatter
# fig = plt.figure()
# ax = fig.add_subplot(111)
# line, = ax.plot(x,y, ls="", marker="o", markersize=0.5)

# # create the annotations box
# image = Image.open(image_paths[0])
# im = OffsetImage(image, zoom=5)
# xybox=(50., 50.)
# ab = AnnotationBbox(im, (0,0), xybox=xybox, xycoords='data',
#         boxcoords="offset points",  pad=0.3,  arrowprops=dict(arrowstyle="->"))
# # add it to the axes and make it invisible
# ax.add_artist(ab)
# ab.set_visible(False)

# def hover(event):
#     # if the mouse is over the scatter points
#     if line.contains(event)[0]:
#         # find out the index within the array from the event
#         ind, = line.contains(event)[1]["ind"]
#         # get the figure size
#         w,h = fig.get_size_inches()*fig.dpi
#         ws = (event.x > w/2.)*-1 + (event.x <= w/2.) 
#         hs = (event.y > h/2.)*-1 + (event.y <= h/2.)
#         # if event occurs in the top or right quadrant of the figure,
#         # change the annotation box position relative to mouse.
#         ab.xybox = (xybox[0]*ws, xybox[1]*hs)
#         # make annotation box visible
#         ab.set_visible(True)
#         # place it at the position of the hovered scatter point
#         ab.xy =(x[ind], y[ind])
#         # set the image corresponding to that point
#         #im.set_data(arr[ind,:,:])
#         image=Image.open(image_paths[ind])
#         im.set_data(image)
#     else:
#         #if the mouse is not over a scatter point
#         ab.set_visible(False)
#     fig.canvas.draw_idle()

# # add callback for mouse moves
# fig.canvas.mpl_connect('motion_notify_event', hover)           
# plt.show()

In [ ]:
counter=0
for classif in Classification:
    if 1==int(classif[0]):
        counter+=1

len(Classification)/counter